<a href="https://colab.research.google.com/github/yonedahayato/stock_strategy/blob/feature%2F49/Numerai/my_Numerai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/yonedahayato/stock_strategy.git

Cloning into 'stock_strategy'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1911 (delta 82), reused 119 (delta 48), pack-reused 1744
Receiving objects: 100% (1911/1911), 1.94 MiB | 4.14 MiB/s, done.
Resolving deltas: 100% (1162/1162), done.


In [2]:
%cd stock_strategy/
!git checkout feature/49
!git fetch
!git rebase
%cd Numerai/

/content/stock_strategy
Branch 'feature/49' set up to track remote branch 'feature/49' from 'origin'.
Switched to a new branch 'feature/49'
First, rewinding head to replay your work on top of it...
Fast-forwarded feature/49 to refs/remotes/origin/feature/49.
/content/stock_strategy/Numerai


In [3]:
!pip install --upgrade pip
!pip install -r requirements.txt

     |████████████████████████████████| 1.5MB 7.3MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [4]:
from glob import glob
import numpy as np
import numerapi
import os
import pandas as pd

from mylogger import mylogger


In [5]:
from datasets import Features, print_log
class Datasets(object):
    """Datasets class

    データセットに関する処理をまとめる

    """
    def __init__(self, dir_="./dataset"):
        """__init__ func

        APIの準備
        パラメータの設置
        ディレクトリの作成

        Args:
            dir_ (str): データセットを保存するパス
        """
        self.napi = numerapi.NumerAPI(verbosity="info")
        self.dir_ = dir_

        os.makedirs(dir_, exist_ok=True)

    def print_log(self, message):
        print_log(message)

    def download(self):
        """download func

        numerapi を使ってデータをdownloadする

        """
        self.napi.download_current_dataset(unzip=True, dest_path=self.dir_)

    def load(self, test=False):
        """load func

        CSVデータをロードする

        Args:
            test (bool): テストデータかどうか

        Return:
            pandas.core.frame.DataFrame: 読み込んだデータ
        """
        files = os.listdir(path="{}/".format(self.dir_))
        dataset_names = [f for f in files if os.path.isdir(os.path.join(self.dir_, f))]
        sorted(dataset_names)
        base_path = os.path.join(self.dir_, "{}/numerai_{}_data.csv")
        if test:
            data_path = base_path.format(dataset_names[0], "tournament")
        else:
            data_path = base_path.format(dataset_names[0], "training")

        self.print_log(data_path)
        self.row_data = pd.read_csv(data_path)

        return self.row_data

    def static(self):
        """static func

        データの統計情報を表示する

        """
        self.print_log("row: {}, colmun: {}".format(self.row_data.shape[0], self.row_data.shape[1]))
        self.print_log(self.row_data.head())

        self.features = Features(self.row_data.columns)
 
        # 平均の計算
        mean_df = self.row_data.mean()
        mean_df.plot(legend=False, kind='hist', title="各列の平均のヒストグラム")


In [6]:
# from datasets import Datasets
datasets = Datasets()
datasets.download()
data_df = datasets.load()

./dataset/numerai_dataset_220.zip:  99%|█████████▉| 378M/381M [00:23<00:00, 19.1MB/s]2020-07-15 12:39:06,753 INFO numerapi.base_api: unzipping file...
./dataset/numerai_dataset_220.zip: 381MB [00:39, 19.1MB/s]                           [I 200715 12:40:00 datasets:8] ./dataset/numerai_dataset_220/numerai_training_data.csv


In [7]:
# datasets.static()

In [8]:
!pip install xgboost

In [9]:
from xgboost import XGBRegressor

feature_names = [f for f in datasets.row_data.columns if "feature" in f]
model = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1)
model.fit(datasets.row_data[feature_names], datasets.row_data["target_kazutsugi"])

[12:40:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=2000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [10]:
test_data = datasets.load(test=True)

[I 200715 13:18:32 datasets:8] ./dataset/numerai_dataset_220/numerai_tournament_data.csv


In [ ]:
predictions = model.predict(test_data[feature_names])

In [ ]:
ranked_prediction = training_data["prediction"].rank(pct=True, method="first")
correlation = np.corrcoef(training_data["target"], ranked_prediction)[0, 1]